In [1]:
suppressPackageStartupMessages({
    library(chromVAR)
    library(motifmatchr)
    library(SummarizedExperiment)
    library(Matrix)
    library('cicero')
})

In [2]:
library(BiocParallel)
register(MulticoreParam(20))

# chromVAR

In [ ]:
## Rscript
# Rscript chromVAR.R -d data/Hematopoiesis-All/data_filtered -p data/Hematopoiesis-All/Hematopoiesis-All_filtered_peak.bed -g hg19 -n hema_all_f -o result/Hematopoiesis-All/chromVAR_f

In [5]:
library(BSgenome.Hsapiens.UCSC.hg19)
genome = BSgenome.Hsapiens.UCSC.hg19
species = 'Homo sapiens'

Loading required package: BSgenome
Loading required package: Biostrings
Loading required package: XVector

Attaching package: 'Biostrings'

The following object is masked from 'package:DelayedArray':

    type

The following object is masked from 'package:base':

    strsplit

Loading required package: rtracklayer


In [7]:
motifs <- getJasparMotifs(species=species)

In [6]:
#data_file = 'data//Hematopoiesis-All/data'
#peak_file = 'data//Hematopoiesis-All/Hematopoiesis-All_peak.bed'
data_file = 'data//Hematopoiesis-All/data_filtered'
peak_file = 'data//Hematopoiesis-All/Hematopoiesis-All_filtered_peak.bed'

dir.create(outdir, recursive=TRUE, showWarnings=FALSE)

#counts = read.table(data_file, row.names=1, header=T)
counts = read_count(data_file)
peaks = getPeaks(peak_file, sort_peaks=T) 

peak_index = as.data.frame(peaks)
peak_index = paste0(peak_index$seqnames, ':' ,peak_index$start-1, '-', peak_index$end)

index = row.names(counts)%in%peak_index
counts = counts[index,]

In [ ]:
dev = apply_chromVAR(counts, peaks, motifs, genome)

In [10]:
outdir = 'result/Hematopoiesis-All/chromVAR_f'
name = 'hema_all_f'
write.table(deviationScores(dev), paste0(outdir, '/', name, '_dev.txt'), quote=F, sep='\t')
var = computeVariability(dev)
write.table(var, paste0(outdir, '/',name, '_var.txt'), quote=F, sep='\t')

In [12]:
saveRDS(dev, "result/Hematopoiesis-All/chromVAR_f/dev.rds")

# Cicero

In [3]:
suppressPackageStartupMessages({
    library(BSgenome.Hsapiens.UCSC.hg19)
    library(TxDb.Hsapiens.UCSC.hg19.knownGene)
    library(org.Hs.eg.db)
    library(cicero)
    library(magrittr)
    })

txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
orgdb <- org.Hs.eg.db
bsgenome <- BSgenome.Hsapiens.UCSC.hg19

In [17]:
# read in matrix data using the Matrix package
indata <- Matrix::readMM("data/Hematopoiesis-All/data/count.mtx") 
# binarize the matrix
indata@x[indata@x > 0] <- 1

# format cell info
cellinfo <- read.table("data/Hematopoiesis-All/data/barcodes.txt",comment.char = "")
row.names(cellinfo) <- cellinfo$V1
names(cellinfo) <- "cells"

# format peak info
peakinfo <- read.table("data/Hematopoiesis-All/Hematopoiesis-All_peak.bed")
names(peakinfo) <- c("chr", "bp1", "bp2")
peakinfo$site_name <- paste(peakinfo$chr, peakinfo$bp1, peakinfo$bp2, sep="_")
row.names(peakinfo) <- peakinfo$site_name

row.names(indata) <- row.names(peakinfo)
colnames(indata) <- row.names(cellinfo)

# make CDS
fd <- methods::new("AnnotatedDataFrame", data = peakinfo)
pd <- methods::new("AnnotatedDataFrame", data = cellinfo)
input_cds <-  suppressWarnings(newCellDataSet(indata,
                            phenoData = pd,
                            featureData = fd,
                            expressionFamily=VGAM::binomialff(),
                            lowerDetectionLimit=0))
input_cds@expressionFamily@vfamily <- "binomialff"
input_cds <- monocle::detectGenes(input_cds)

#Ensure there are no peaks included with zero reads
input_cds <- input_cds[Matrix::rowSums(exprs(input_cds)) != 0,] 


In [ ]:
cells = read.table('data/Hematopoiesis-All/GSE129785_scATAC-Hematopoiesis-All.cell_barcodes.txt', 
                   comment.char = "", head=1)
dimred <- data.frame(row.names = cells$Group_Barcode, 
                     cells$UMAP1, 
                     cells$UMAP2)

In [ ]:
set.seed(2017)
input_cds <- detectGenes(input_cds)
input_cds <- estimateSizeFactors(input_cds)

In [19]:
saveRDS(input_cds, "result/Hematopoiesis-All/hema_all_input.rds")

In [2]:
input_cds <- readRDS("result/Hematopoiesis-All/hema_all_input.rds")

In [ ]:
cicero_cds  <- make_cicero_cds(input_cds, k = 50, reduced_coordinates = dimred[colnames(input_cds),])

In [5]:
object.size(input_cds)

129578392 bytes

In [18]:
object.size(input_cds)

129578392 bytes

In [33]:
cicero_cds  <- make_cicero_cds(input_cds, k = 50, reduced_coordinates = dimred[colnames(input_cds),])

Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.0384606816936211
Median shared cells bin-bin: 0


ERROR: Error in asMethod(object): Cholmod error 'problem too large' at file ../Core/cholmod_dense.c, line 105


In [6]:
input_cds_f=input_cds[Matrix::rowSums(exprs(input_cds)) >= 1000

In [7]:
object.size(input_cds_f)

38079080 bytes

In [8]:
saveRDS(input_cds_f, "result/Hematopoiesis-All/hema_all_input_f.rds")

In [43]:
set.seed(2017)
input_cds_f <- detectGenes(input_cds_f)
input_cds_f <- estimateSizeFactors(input_cds_f)

In [44]:
cicero_cds_f  <- make_cicero_cds(input_cds_f, k = 50, reduced_coordinates = dimred[colnames(input_cds_f),])

Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.0384606816936211
Median shared cells bin-bin: 0


In [45]:
saveRDS(cicero_cds_f, "result/Hematopoiesis-All/hema_all_cicero_f.rds")

In [3]:
cicero_cds_f <- readRDS("result/Hematopoiesis-All/hema_all_cicero_f.rds")

In [54]:
# choose genome region
bsgenome <- BSgenome.Hsapiens.UCSC.hg19
chromSizes <- seqlengths(bsgenome)[paste0("chr",c(1:22,"X"))]
genome <- data.frame(names(chromSizes),chromSizes)
rownames(genome) <- NULL

In [ ]:
data("human.hg19.genome")
sample_genome <- subset(human.hg19.genome, V1 == "chr1")
conns <- run_cicero(cicero_cds_f, sample_genome) # Takes a few minutes to run

In [11]:
write.table(conns, 'result//Hematopoiesis-All/hema_all_f_conns.txt',quote = FALSE, sep='\t', row.names = FALSE)

In [4]:
conns = read.table( 'result//Hematopoiesis-All/hema_all_f_conns.txt', sep='\t', header = 1)
input_cds_f = readRDS('result//Hematopoiesis-All/hema_all_input_f.rds')

In [8]:
tssWindow <- 2500
flank <- 250*10^3
corCutOff <- 0.35

#Annotate CDS
message("Annotating Cell Data Set...")
genes <- getTxDbGenes(txdb=txdb,orgdb=orgdb)
names(genes) <- genes$symbol
genes <- resize(genes, 1, "start") %>% resize(tssWindow * 2 + 1, "center")
geneDF <- data.frame(chromosome=seqnames(genes),start=start(genes),end=end(genes), gene=genes$symbol)
obj <- annotate_cds_by_site(input_cds_f, geneDF)

#Prepare for Co-Accessibility
nSites <- Matrix::colSums(assayData(obj)$exprs)
names(nSites) <- row.names(pData(obj))


Annotating Cell Data Set...


In [9]:
#Cicero with Correlations
message("Calculating normalized gene activities...")
ciceroGA <- normalize_gene_activities(build_gene_activity_matrix(obj, conns, coaccess_cutoff = corCutOff), nSites)

Calculating normalized gene activities...


In [ ]:
library('Matrix')

In [ ]:
writeMM(ciceroGA,file='result//Hematopoiesis-All/hema_all_f_ciceroGA.txt')
write.table(rownames(ciceroGA),'result//Hematopoiesis-All/hema_all_f_ciceroGA_genes.txt',
           quote = FALSE)
write.table(colnames(ciceroGA),'result//Hematopoiesis-All/hema_all_f_ciceroGA_barcodes.txt',
           quote = FALSE)

In [11]:
dim(ciceroGA)

[1] 13541 63882

In [ ]:
head(ciceroGA)

In [20]:
library(SummarizedExperiment)
seCicero <- SummarizedExperiment(
	assays = SimpleList(gA = ciceroGA),
	rowRanges = genes[rownames(ciceroGA),],
	colData = mdata
)

seCiceroLog <- SummarizedExperiment(
	assays = SimpleList(logGA = log2(10^6 * ciceroGA + 1)),
	rowRanges = genes[rownames(ciceroGA),],
	colData = mdata
)

#Save Output
#saveRDS(connections, "results/Peaks-Co-Accessibility.rds")
#saveRDS(seCicero, "results/Cicero-Gene-Activity.rds")
#saveRDS(seCiceroLog, "results/Cicero-Log2-Gene-Activity.rds")

Loading required package: DelayedArray
Loading required package: matrixStats

Attaching package: 'matrixStats'

The following objects are masked from 'package:Biobase':

    anyMissing, rowMedians

Loading required package: BiocParallel

Attaching package: 'DelayedArray'

The following objects are masked from 'package:matrixStats':

    colMaxs, colMins, colRanges, rowMaxs, rowMins, rowRanges

The following object is masked from 'package:Biostrings':

    type

The following objects are masked from 'package:base':

    aperm, apply



ERROR: Error in is(colData, "DataFrame"): object 'mdata' not found


# trajectory

In [ ]:
#----------------------------
# Get Inputs
#----------------------------
se <- readRDS("data/Hematopoiesis-All/scATAC_Heme_All_SummarizedExperiment.final.rds")
trajectory <- paste0("Cluster",c(1,4,5,6,7,14,15,16))

#Align single cells to Trajectory
df <- data.frame(row.names = colnames(se), x = colData(se)$UMAP1, y = colData(se)$UMAP2, Group = colData(se)$Clusters)
trajAligned <- alignTrajectory(df, trajectory)

df2 <- trajAligned[[1]]
dfT <- trajAligned[[2]]

pdf("result/Hematopoiesis-All/trajectory/UMAP-PseudoTime.pdf")
library(ggplot2)
ggplot(df2, aes(x,y,color=pseudotime)) + geom_point() +
    theme_bw() + viridis::scale_color_viridis() +
    geom_path(data=data.frame(dfT), aes(x,y,color=NULL), size= 1, 
        arrow = arrow(type = "open", angle = 30, length = unit(0.1, "inches")))
dev.off()
saveRDS(trajAligned, "result/Hematopoiesis-All/trajectory/Aligned-Trajectory.rds")
write.table(trajAligned$trajectory, 'result/Hematopoiesis-All/trajectory/trajectory_stats.txt', sep='\t', quote=FALSE)

#Significance of Trajectory Order
clustMeans <- edgeR::cpm(groupSums(assay(se), colData(se)$Clusters, sparse = TRUE),log=TRUE,prior.count=3)
trajStats <- trajectoryStats(clustMeans, trajectory, nSim = 500, nFeatures = 10000)
rankMat <- trajStats$rankTMat
rankDF <- lapply(seq_len(ncol(rankMat)), function(x){
    data.frame(names=names(rankMat[,x]),ranks=rankMat[,x],t=x)
}) %>% Reduce("rbind",.)
rankV <- lapply(seq_len(ncol(rankMat)), function(x){
    data.frame(ranks=rankMat[rev(trajectory)[x+1],x],t=x)
}) %>% Reduce("rbind",.)

pdf("result/Hematopoiesis-All/trajectory/Trajectory_Plot_Distance.pdf", useDingbats = FALSE, width = 12, height = 12)
ggplot(rankDF, aes(t,ranks,color=names)) + 
    geom_point(size = 8) + 
    geom_point(data = rankV, aes(color=NULL), color = "black", size = 2) + 
    geom_line(data = rankV, aes(color=NULL), color = "black", size = 1, 
        lty = "dashed", arrow = arrow(type = "open", length = unit(0.2, "inches"))) + 
    theme_bw() + 
    ylab("Trajectory Distance (Dist logCPM)") + xlab(paste0("Trajectory 5000 Sim Pval = ",trajStats$pvalMean)) + 
    ggtitle(paste0(rev(trajectory),collapse=" -> "))
dev.off()

out <- list(trajStats = trajStats, rankDF = rankDF, rankV = rankV)
saveRDS(out, "result/Hematopoiesis-All/trajectory/Trajectory-Stats.rds")


In [2]:
trajAligned=readRDS("result/Hematopoiesis-All/trajectory/Aligned-Trajectory.rds")

In [10]:
hema=readRDS("data/Hematopoiesis-All/scATAC_Heme_All_SummarizedExperiment.final.rds")

In [12]:
hema

class: RangedSummarizedExperiment 
dim: 571400 63882 
metadata(0):
assays(1): counts
rownames: NULL
rowData names(3): name SYMBOL GC
colnames(63882): SUHealthy_BM_B1_50 SUHealthy_BM_B1_51 ... pDC_76
  pDC_80
colData names(9): UMAP1 UMAP2 ... Internal_Name Group_Barcode

In [7]:
write.table(trajAligned$trajectory, 'result/Hematopoiesis-All/trajectory/trajectory_stats.txt', sep='\t', quote=FALSE)

## CD34

In [2]:
source('trajectory.R')

[1] "Done"


In [ ]:
se <- readRDS("data/Hematopoiesis-CD34/scATAC_CD34_BM_SummarizedExperiment.final.rds")

In [9]:
se

class: RangedSummarizedExperiment 
dim: 571389 18489 
metadata(0):
assays(1): counts
rownames: NULL
rowData names(3): name SYMBOL GC
colnames(18489): SUHealthy_BM_B1_50 SUHealthy_BM_B1_52 ... pDC_76
  pDC_80
colData names(9): UMAP1 UMAP2 ... Internal_Name Group_Barcode

In [8]:
write.table(colData(se),'data/Hematopoiesis-CD34/cells.txt', sep='\t', quote=FALSE, row.names = FALSE)

In [11]:
scale_umap=read.table('result/Hematopoiesis-CD34/scale_cluster/umap.txt', sep=' ')
cluster=read.table('result/Hematopoiesis-CD34/scale_cluster/cluster_assignments.txt', sep='\t', comment.char = "")

In [16]:
colData(se)$UMAP1=scale_umap$V1
colData(se)$UMAP2=scale_umap$V2
colData(se)$scale_cluster=cluster$V2

In [30]:
saveRDS(se, 'result/Hematopoiesis-CD34/scATAC_CD34_BM_SummarizedExperiment.scale.rds')

In [57]:
trajectory=c(13,4,14,11,12,3)
lineage='B'

In [55]:
trajectory=c(13,4,0,5,10,15)
lineage='pDC1'

In [53]:
trajectory=c(13,4,0,5,7)
lineage='pDC2'

In [61]:
trajectory=c(13,4,9,6)
lineage='Meg'

In [62]:
#traj <- function(trajectory, lineage){
    #Align single cells to Trajectory
    df <- data.frame(row.names = colnames(se), x = colData(se)$UMAP1, y = colData(se)$UMAP2, Group = colData(se)$scale_cluster)
    trajAligned <- alignTrajectory(df, trajectory)

    df2 <- trajAligned[[1]]
    dfT <- trajAligned[[2]]

    png(paste0("result/Hematopoiesis-CD34/trajectory/",lineage, "_UMAP-PseudoTime.png"))
    #library(ggplot2)
    ggplot(df2, aes(x,y,color=pseudotime)) + geom_point() +
        theme_bw() + viridis::scale_color_viridis() +
        geom_path(data=data.frame(dfT), aes(x,y,color=NULL), size= 1, 
            arrow = arrow(type = "open", angle = 30, length = unit(0.1, "inches")))
    dev.off()
    saveRDS(trajAligned, paste0("result/Hematopoiesis-CD34/trajectory/",lineage,"_Aligned-Trajectory.rds"))
    write.table(trajAligned$trajectory, paste0("result/Hematopoiesis-CD34/trajectory/",lineage,"_trajectory_stats.txt"), sep='\t', quote=FALSE)

#}


png 
  2

In [39]:
traj(c(13,4,14,11,12,3),'B2')

In [36]:
traj(c(13,4,0,5,10,15), 'pDC1')

In [48]:
traj(c(13,4,0,5,7), 'pDC2')

# TME

In [2]:
ga <- readRDS('data/TME-All/Log2_Gene_Activity_TME_All_SummarizedExperiment.final.rds')

In [3]:
ga

class: RangedSummarizedExperiment 
dim: 18993 37818 
metadata(0):
assays(1): logGA
rownames(18993): A1BG A1BG-AS1 ... ZZEF1 ZZZ3
rowData names(2): gene_id symbol
colnames(37818): SU001_Tcell_Post_133 SU001_Tcell_Post_134 ...
  SU005_Total_Post_596 SU005_Total_Post_597
colData names(9): UMAP1 UMAP2 ... Internal_Name Group_Barcode

In [6]:
write.table(rownames(ga),'data/TME-All/cicero_ga_genes.txt',quote = FALSE, row.names=FALSE)
write.table(colnames(ga),'data/TME-All/cicero_ga_cells.txt',quote = FALSE, row.names=FALSE)

In [17]:
write.table(as.matrix(assays(ga)$logGA),file='data/TME-All/gene_activity_cicero_ga.txt')

In [31]:
tf <- readRDS('data/TME-All/chromVAR_TME_All_SummarizedExperiment.final.rds')

In [ ]:
t

In [19]:
tf

class: chromVARDeviations 
dim: 1764 37818 
metadata(3): Variability SummarizedExperiment motifMatches
assays(2): deviations z
rownames(1764): TFAP2B_1 TFAP2B_2 ... TBX22_1763 TBX22_1764
rowData names(3): name fractionMatches fractionBackgroundOverlap
colnames(37818): SU001_Tcell_Post_133 SU001_Tcell_Post_134 ...
  SU005_Total_Post_596 SU005_Total_Post_597
colData names(9): UMAP1 UMAP2 ... Internal_Name Group_Barcode

In [20]:
write.table(as.matrix(assays(tf)$deviations),file='data/TME-All/tf_deviations.txt')

In [21]:
write.table(as.matrix(assays(tf)$z),file='data/TME-All/tf_var.txt')

# CNV

In [ ]:
suppressPackageStartupMessages(source('CNV.R'))

In [4]:
blacklist <- import.bed("../data/hg19.blacklist.bed")

In [5]:
blacklist

GRanges object with 411 ranges and 2 metadata columns:
        seqnames            ranges strand |                    name     score
           <Rle>         <IRanges>  <Rle> |             <character> <numeric>
    [1]     chr1     564450-570371      * | High_Mappability_island      1000
    [2]     chr1     724137-727043      * |        Satellite_repeat      1000
    [3]     chr1     825007-825115      * |                BSR/Beta      1000
    [4]     chr1   2583335-2634374      * |  Low_mappability_island      1000
    [5]     chr1   4363065-4363242      * |                (CATTC)n      1000
    ...      ...               ...    ... .                     ...       ...
  [407]     chrY 28555027-28555353      * |                    TAR1      1000
  [408]     chrY 28784130-28819695      * |        Satellite_repeat      1000
  [409]     chrY 58819368-58917648      * |                (CATTC)n      1000
  [410]     chrY 58971914-58997782      * |                (CATTC)n      1000
  [411]  

In [ ]:
windows <- makeWindows(genome = BSgenome.Hsapiens.UCSC.hg19, blacklist = blacklist)

In [8]:
windows

GRanges object with 3296 ranges and 7 metadata columns:
        seqnames            ranges strand |        wSeq    wStart      wEnd
           <Rle>         <IRanges>  <Rle> | <character> <integer> <integer>
     w1     chr1          1-564449      * |        chr1         1  10000000
     w1     chr1     570372-724136      * |        chr1         1  10000000
     w1     chr1     727044-825006      * |        chr1         1  10000000
     w1     chr1    825116-2583334      * |        chr1         1  10000000
     w2     chr1   2000001-2583334      * |        chr1   2000001  12000000
    ...      ...               ...    ... .         ...       ...       ...
  w1436     chrY 40000001-50000000      * |        chrY  40000001  50000000
  w1437     chrY 42000001-52000000      * |        chrY  42000001  52000000
  w1438     chrY 44000001-54000000      * |        chrY  44000001  54000000
  w1439     chrY 46000001-56000000      * |        chrY  46000001  56000000
  w1440     chrY 48000001-580000

In [24]:
write.table(windows, "result/TME-All/windows.txt", sep='\t', quote=FALSE)

In [14]:
saveRDS(windows, "result/TME-All/windows.rds")

In [10]:
frag_bed=import.bed('data/TME-All/SU010_pre.fragment.tsv.gz')
frag_bed$RG <- frag_bed$name

In [ ]:
cnaObj <- scCNA(windows, frag_bed, neighbors = 100, LFC = 1.5, FDR = 0.1, force = TRUE, 
                remove = paste0("chr",c(1,2,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y','M')))

In [17]:
cnaObj

class: RangedSummarizedExperiment 
dim: 274 260844 
metadata(0):
assays(8): CNA counts ... bdgMean bdgSd
rownames: NULL
rowData names(6): name effectiveLength ... AT N
colnames(260844): GCAGCCAAGAACTAAC-1 CCACGTTCACACATGT-1 ...
  AGTTTGGCACATGATC-1 AGCCTCTGTAGTCCAT-1
colData names(0):

In [30]:
assays(cnaObj)$z[1:5,1:5]

GCAGCCAAGAACTAAC-1,CCACGTTCACACATGT-1,AACCTTTGTAAACCCT-1,ACAAACCGTTACGAAA-1,AGTGTACAGAAATGGG-1
0.13209790,-0.3924358,0.6274611,-0.2875253,2.074265
0.04589254,-0.4301763,0.7042428,-0.6438970,2.252947
-0.55693049,-0.4301763,0.7042428,-0.6438970,2.338119
-0.55693049,-0.4301763,0.7042428,-0.5105120,3.206876
-0.55693049,-0.5451190,-0.6021806,-0.5105120,3.025633


In [18]:
saveRDS(cnaObj, "result/TME-All/SU010pre_cnv_3-6.rds")

In [23]:
rowData(cnaObj)

DataFrame with 274 rows and 6 columns
           name effectiveLength percentEffectiveLength        GC        AT
    <character>       <integer>              <numeric> <numeric> <numeric>
1          w239        10000000                    100 0.4145635 0.5854365
2          w240        10000000                    100 0.4326718 0.5673282
3          w241        10000000                    100 0.4400118 0.5599882
4          w242        10000000                    100 0.4443293 0.5556707
5          w243        10000000                    100 0.4311841 0.5688159
...         ...             ...                    ...       ...       ...
270        w579        10000000                    100 0.3810121 0.6189879
271        w580        10000000                    100 0.3814109 0.6185891
272        w581         9999793               99.99793 0.3971409 0.6028384
273        w582         9999793               99.99793  0.396502 0.6034773
274        w583         9999793               99.99793 0.39924

In [28]:
write.table(assays(cnaObj)$z,'result//TME-All/CNA_z.txt',sep='\t', quote=FALSE)
write.table(rowData(cnaObj),'result//TME-All/CNA_window.txt',sep='\t', quote=FALSE)
write.table(assays(cnaObj)$CNA,'result//TME-All/CNA.txt',sep='\t', quote=FALSE)